In [785]:
import sqlite3
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

In [786]:
con=sqlite3.connect('database.sqlite')

In [787]:
df=pd.read_sql_query("SELECT * FROM Player_Attributes",con)

In [788]:
df.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [789]:
df.shape

(183978, 42)

In [790]:
df.columns

Index(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating',
       'potential', 'preferred_foot', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning',
       'gk_reflexes'],
      dtype='object')

In [791]:
df.isnull().values.any()

True

In [778]:
#drpcol=['date','id', 'player_fifa_api_id', 'player_api_id']
drpcol=['date','id','player_api_id']

In [794]:
df2=df.groupby(['player_fifa_api_id'])
df2.head()
#df2=df.dropna()
#df2.drop(drpcol,axis=1,inplace=True)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
5,6,189615,155782,2016-04-21 00:00:00,74.0,76.0,left,high,medium,80.0,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
6,7,189615,155782,2016-04-07 00:00:00,74.0,76.0,left,high,medium,80.0,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
7,8,189615,155782,2016-01-07 00:00:00,73.0,75.0,left,high,medium,79.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
8,9,189615,155782,2015-12-24 00:00:00,73.0,75.0,left,high,medium,79.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
9,10,189615,155782,2015-12-17 00:00:00,73.0,75.0,left,high,medium,79.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0


In [767]:
df2.shape

(180354, 39)

In [762]:
df2.head()

,player_fifa_api_id,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,218353,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,218353,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,218353,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,218353,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,218353,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [763]:
cat_data=df2.dtypes[df2.dtypes=='object'].index
cat_data

Index(['preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], dtype='object')

In [721]:
lc=[3,4,5]

In [722]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder = LabelEncoder()
for i in lc:
    df2.iloc[:, i] = labelencoder.fit_transform(df2.iloc[:, i])

C:\Users\abhinav\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [723]:
df2.head()

,player_fifa_api_id,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,218353,67.0,71.0,1,4,14,49.0,44.0,71.0,61.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,218353,67.0,71.0,1,4,14,49.0,44.0,71.0,61.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,218353,62.0,66.0,1,4,14,49.0,44.0,71.0,61.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,218353,61.0,65.0,1,4,14,48.0,43.0,70.0,60.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,218353,61.0,65.0,1,4,14,48.0,43.0,70.0,60.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [724]:
df2.columns

Index(['player_fifa_api_id', 'overall_rating', 'potential', 'preferred_foot',
       'attacking_work_rate', 'defensive_work_rate', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve',
       'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration',
       'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power',
       'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
       'interceptions', 'positioning', 'vision', 'penalties', 'marking',
       'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes'],
      dtype='object')

From above graphs we can derive that column preferred_foot is not having much impact on overall ratings.

In [725]:
crr=df2.corr().abs()
crr

,player_fifa_api_id,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
player_fifa_api_id,1.000000,0.278703,0.021252,0.008626,0.043737,0.007222,0.065631,0.029836,0.103500,0.065311,...,0.163099,0.175255,0.075568,0.071128,0.055218,0.092945,0.138844,0.248222,0.140925,0.131531
overall_rating,0.278703,1.000000,0.765435,0.001417,0.129166,0.046834,0.357320,0.330079,0.313324,0.458243,...,0.431493,0.392715,0.132185,0.163986,0.128054,0.027675,0.006717,0.028799,0.008029,0.007804
potential,0.021252,0.765435,1.000000,0.003513,0.114670,0.012218,0.277284,0.287838,0.206063,0.382538,...,0.379278,0.315207,0.054094,0.082073,0.063284,0.012283,0.005865,0.092299,0.004472,0.004936
preferred_foot,0.008626,0.001417,0.003513,1.000000,0.032065,0.013930,0.174033,0.012004,0.004765,0.073302,...,0.039395,0.016476,0.095221,0.091167,0.098139,0.074524,0.077679,0.044258,0.075666,0.075233
attacking_work_rate,0.043737,0.129166,0.114670,0.032065,1.000000,0.212135,0.247827,0.236444,0.103900,0.182351,...,0.187983,0.179174,0.008189,0.007133,0.011389,0.165154,0.160999,0.109389,0.162644,0.163737
defensive_work_rate,0.007222,0.046834,0.012218,0.013930,0.212135,1.000000,0.027891,0.018891,0.094160,0.074092,...,0.042325,0.035335,0.085628,0.090396,0.088061,0.079904,0.079708,0.054490,0.078894,0.078783
crossing,0.065631,0.357320,0.277284,0.174033,0.247827,0.027891,1.000000,0.576896,0.368956,0.790323,...,0.693978,0.574208,0.234886,0.285018,0.274673,0.604567,0.595646,0.356728,0.597742,0.601696
finishing,0.029836,0.330079,0.287838,0.012004,0.236444,0.018891,0.576896,1.000000,0.373459,0.580245,...,0.652376,0.726234,0.285416,0.230453,0.262144,0.479370,0.465135,0.292349,0.470758,0.473302
heading_accuracy,0.103500,0.313324,0.206063,0.004765,0.103900,0.094160,0.368956,0.373459,1.000000,0.548435,...,0.336472,0.431291,0.460831,0.480054,0.441134,0.665600,0.649145,0.402865,0.648981,0.652494
short_passing,0.065311,0.458243,0.382538,0.073302,0.182351,0.074092,0.790323,0.580245,0.548435,1.000000,...,0.766401,0.612511,0.349578,0.415427,0.380148,0.694111,0.689874,0.422659,0.691030,0.693260


In [726]:
#import seaborn as sns

From the correlation we can find that preferred_foot,gk_diving
,gk_handling,gk_kicking,gk_positioning,gk_reflexes

In [727]:
#drpcol2=['preferred_foot','gk_reflexes','gk_positioning','gk_handling','gk_kicking','gk_diving']

In [728]:
df2.drop(drpcol2,axis=1,inplace=True)

In [729]:
df2.head()

,player_fifa_api_id,overall_rating,potential,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,...,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle
0,218353,67.0,71.0,4,14,49.0,44.0,71.0,61.0,44.0,...,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0
1,218353,67.0,71.0,4,14,49.0,44.0,71.0,61.0,44.0,...,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0
2,218353,62.0,66.0,4,14,49.0,44.0,71.0,61.0,44.0,...,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0
3,218353,61.0,65.0,4,14,48.0,43.0,70.0,60.0,43.0,...,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0
4,218353,61.0,65.0,4,14,48.0,43.0,70.0,60.0,43.0,...,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0


Removing the outliers using z score

In [730]:
from scipy import stats
import numpy as np

In [731]:
z=np.abs(stats.zscore(df2))
z

array([[0.97557003, 0.2326883 , 0.37670587, ..., 0.85941725, 0.86740492,
        0.97118884],
       [0.97557003, 0.2326883 , 0.37670587, ..., 0.85941725, 0.86740492,
        0.97118884],
       [0.97557003, 0.94413529, 1.13635982, ..., 0.85941725, 0.72784556,
        0.97118884],
       ...,
       [1.22040029, 1.19020567, 0.99067124, ..., 0.69523066, 0.90034696,
        0.83497124],
       [1.22040029, 1.33249507, 1.14260203, ..., 0.88367283, 0.85382717,
        0.83497124],
       [1.22040029, 1.61707386, 1.14260203, ..., 0.4125674 , 0.85382717,
        0.83497124]])

In [732]:
df2 = df2[(z <=3).all(axis=1)]
df2.head()

,player_fifa_api_id,overall_rating,potential,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,...,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle
0,218353,67.0,71.0,4,14,49.0,44.0,71.0,61.0,44.0,...,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0
1,218353,67.0,71.0,4,14,49.0,44.0,71.0,61.0,44.0,...,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0
2,218353,62.0,66.0,4,14,49.0,44.0,71.0,61.0,44.0,...,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0
3,218353,61.0,65.0,4,14,48.0,43.0,70.0,60.0,43.0,...,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0
4,218353,61.0,65.0,4,14,48.0,43.0,70.0,60.0,43.0,...,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0


In [733]:
df2.shape

(164467, 33)

In [734]:
df2.columns

Index(['player_fifa_api_id', 'overall_rating', 'potential',
       'attacking_work_rate', 'defensive_work_rate', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve',
       'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration',
       'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power',
       'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
       'interceptions', 'positioning', 'vision', 'penalties', 'marking',
       'standing_tackle', 'sliding_tackle'],
      dtype='object')

In [735]:
featcol=[  'potential', 'attacking_work_rate',
       'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle']

In [736]:
X=df2[featcol]

In [737]:
y=df2['overall_rating']

In [738]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [739]:
regressor=LinearRegression()

In [740]:
regressor.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [741]:
y_pred=np.array(regressor.predict(X_test))

In [742]:
y_target=np.array(y_test)

In [743]:
rmse=np.sqrt(((y_pred-y_target)**2).mean())
rmse

3.0450634322974777

Using Decision Tree

In [ ]:
from sklearn import tree, metrics, model_selection , preprocessing

In [ ]:
dtree1=tree.DecisionTreeClassifier(criterion='entropy',max_depth=4,random_state=0)
dtree1.fit(X_train,y_train)

In [ ]:
y_pred2=dtree1.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred2)

In [ ]:
accuracy=metrics.accuracy_score(y_test,y_pred2)

In [ ]:
print('The Accuracy of model is= {}'.format(accuracy))